## Tabla de Contenidos
1. [Introducción](#introduccion)
2. [Objetivo](#objetivo)
3. [Datos y Librerías](#datos-y-librerias)
    1. [Librerias](#librerias)
    2. [Datos](#datos)
    3. [Conociendo los datos](#conociendo-los-datos)
    4. [Preparacion de los datos](#preparacion-de-los-datos)
4. [Modelo: Arbol de decision](#arbol-de-decision)
5. [Modelo: Bosque aleatorio](#bosque-aleatorio)
6. [Modelo: Regresion logistica](#regresion-logistica)
7. [Testeo](#testeo)
8. [Conclusión](#conclusion)
9. [Comentarios Generales](#comentarios-generales)

<a id="introduccion"></a>
## Introducción

En este proyecto crearemos un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline:

- Smart.
- Ultra.

Para esto analizaremos la información en base a 3 tipos de modelos:

- Arbol de decisión.
- Bosque aleatorio.
- Regresión logística.

En cada uno de estos modelos veremos y calibraremos sus hiperparámetros para buscar la mayor exactitud posible.

<a id="objetivo"></a>
## Objetivo

Desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline:

- Smart.
- Ultra.

<a id="datos-y-librerias"></a>
## Datos y librerias

<a id="librerias"></a>
### Librerias

In [1]:
# Cargamos las librerias que nos servirán para nuestro análisis
import pandas as pd
from sklearn import set_config
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler 

<a id="datos"></a>
### Datos

In [2]:
# Cargamos los datos de cada tabla por separado

try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

<a id="conociendo-los-datos"></a>
### Conociendo los datos

Hagamos una rápida inspección de los datos para saber con que estamos entrenando a nuestro modelo.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head(10)

calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
5   58.0   344.56      21.0  15823.37         0
6   57.0   431.64      20.0   3738.90         1
7   15.0   132.40       6.0  21911.60         0
8    7.0    43.39       3.0   2538.67         1
9   90.0   665.41      38.0  17358.61         0

Aprovechemos que este DataFrame se compone exclusivamente por números y describamoslo estadísticamente.

In [5]:
df.describe()

calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700000     1.000000
max     244.000000  1632.060000   224.000000  49745.730000     1.000000

<a id="preparacion-de-los-datos"></a>
### Preparacion los datos

Aunque muchas veces los DataFrames tienen datos faltantes, no es el caso esta vez. Eso no significa que no debamos preparar los datos antes de trabajarlos. Para poder entrenar, validar y testear nuestro modelo necesitamos tomar 2 pasos antes. Estos son:

- Dividir los datos para tener los distintos grupos de datos, en este caso dejaremos 20% para validación y otro 20% para testeo, el resto será para entrenar.
- Crear los targets y features para nuestro modelo, considerando entrenamiento, validación y testeo, en este caso consideramos la columna final is_ultra como el target.

In [6]:
# Primero separamos el grupo de testeo
df_rest, df_test = train_test_split(df, test_size=0.2, random_state=54321)

In [7]:
print(df_rest.shape)
print(df_test.shape)

(2571, 5)
(643, 5)


In [8]:
# Luego separamos en entrenamiento y validación
df_train, df_valid = train_test_split(df_rest, test_size=0.25, random_state=54321)

In [9]:
# Creamos targets y features para nuestro modelo
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [10]:
# Veamos los tamaños finales de nuestros DataFrames
print('Entrenamiento:')
print(features_train.shape)
print(target_train.shape)
print()
print('Validación:')
print(features_valid.shape)
print(target_valid.shape)
print()
print('Testeo:')
print(features_test.shape)
print(target_test.shape)

Entrenamiento:
(1928, 4)
(1928,)

Validación:
(643, 4)
(643,)

Testeo:
(643, 4)
(643,)


Como usaremos los mismos datos durante cada modelo, resulta más conveniente hacer esto de manera previa.

<a id="arbol-de-decision"></a>
## Modelo: Arbol de decision

Comenzaremos con el arbol de decisión, en este caso iremos cambiando su profundidad y su cantidad de hojas. Cambiaremos varias variable aprovechando que este modelo tiene una alta velocidad de procesamiento. Esto nos deja jugar un poco más con los detalles.

In [11]:
best_tree = 0
best_depth = 0
best_leaf = 0
score = 0
for depth in range(1, 20): # selecciona el rango del hiperparámetro
    for leaf in range(1,20):
        tree = DecisionTreeClassifier(random_state=54321, max_depth=depth, min_samples_leaf=leaf) # configura el número de árboles
        tree.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
        score = tree.score(features_valid,target_valid) # calcula la puntuación de exactitud en el conjunto de validación
        if score > best_tree:
            best_tree = score # guarda la mejor puntuación de exactitud en el conjunto de validación
            best_depth = depth # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud
            best_leaf = leaf

print("Exactitud del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_tree))

final_tree = DecisionTreeClassifier(random_state=54321, max_depth=best_depth, min_samples_leaf=best_leaf) # cambia n_estimators para obtener el mejor modelo
final_tree.fit(features_train, target_train)

Exactitud del mejor modelo en el conjunto de validación (depth = 7): 0.8429237947122862


DecisionTreeClassifier(max_depth=7, min_samples_leaf=13, random_state=54321)

Resulta que nuestro mejor arbol tiene una profundidad de 7 y 13 hojas en sus ramas. Este modelo nos deja con una exactitud de 84.3%

<a id="bosque-aleatorio"></a>
## Modelo: Bosque aleatorio

Ahora que tenemos un buen arbol, plantemos un bosque, también le daremos espacio para que estos arboles crezcan libres. Aunque esto pueda consumir más tiempo, puede aumentar la exactitud de nuestro modelo. En este caso aprovecharé la potencia de mi maquina.

In [12]:
best_forest = 0
best_est = 0
score=0
best_leaf = 0
best_depth = 0
for est in range(1, 20): # selecciona el rango del hiperparámetro
    for depth in range(1, 20): # selecciona el rango del hiperparámetro
        for leaf in range(1,20):
            forest = RandomForestClassifier(random_state=54321, n_estimators=est, max_depth=depth, min_samples_leaf=leaf) # configura el número de árboles
            forest.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
            score = forest.score(features_valid,target_valid) # calcula la puntuación de exactitud en el conjunto de validación
            if score > best_forest:
                best_forest = score# guarda la mejor puntuación de exactitud en el conjunto de validación
                best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud
                best_leaf = leaf
                best_depth = depth

print("Exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_forest))

final_forest = RandomForestClassifier(random_state=54321, n_estimators=best_est, max_depth=best_depth, min_samples_leaf=best_leaf) # cambia n_estimators para obtener el mejor modelo
final_forest.fit(features_train, target_train)

Exactitud del mejor modelo en el conjunto de validación (n_estimators = 3): 0.8506998444790047


RandomForestClassifier(max_depth=11, min_samples_leaf=3, n_estimators=3,
                       random_state=54321)

Aunque la exactitud terminó aumentando solo un poco, estamos a pasos del 85.1%. Nuestro modelo final tiene 3 arboles, 11 de profundidad y 3 hojas. La gran cantidad de profundidad podría ser evidencia de un sobre ajuste, veremos como se comporta este modelo en la fase de testeo.

Podría decirse que el tiempo de procesamiento necesario no vale la pena para el pequeño aumento obtenido, pero una vez encontrado, este sí resulta mejor.

<a id="regresion-logistica"></a>
## Modelo: Regresion logisctica

Para no dejarlo atrás, también probaremos con una regresión logística, aunque no se espera obtener un mejor modelo que los anteriores.

In [13]:
reg = LogisticRegression(random_state=54321, solver='liblinear')
reg.fit(features_train,target_train)
reg.score(features_valid,target_valid)

0.776049766718507

Este modelo tiene poco espacio para calibrar, aunque se probaron varios solver diferentes, su exactitud a penas llega al 77.6%. Continuaremos con nuestro bosque, pues posee la mayor exactitud.

<a id="testeo"></a>
## Testeo

Considerando que tenemos 2 modelos con exactitudes muy cercanas, haremos testeos para cada uno. Para esto, entrenaremos el modelo que estemos testeando con df_rest, que incluye los datos originales de entrenamiento y de validación. Una vez tengamos el modelo entrenado, lo compararemos con los datos en df_test. Esperamos obtener un alto porcentaje de exactitud, sobre el 84% si todo sale bien.

Primero creemos los features y target para df_rest.

In [14]:
features_rest = df_rest.drop(['is_ultra'], axis=1)
target_rest = df_rest['is_ultra']

In [15]:
final_forest= RandomForestClassifier(max_depth=11, min_samples_leaf=3, n_estimators=3, random_state=54321)
final_forest.fit(features_rest,target_rest)

score = final_forest.score(features_test, target_test)
print(score)

0.7807153965785381


Ha bajado la exactitud de este modelo en 7 puntos porcentuales. Esto probablemente se debe al sobreajuste que este modelo tiene. Probaremos con el modelo del arbol de decisión antes de tomar nuevas medidas.

In [16]:
final_tree = DecisionTreeClassifier(max_depth=7, min_samples_leaf=13, random_state=54321)
final_tree.fit(features_rest, target_rest)

score = final_tree.score(features_test, target_test)
print(score)

0.7729393468118196


Nuevamente ha bajado la exactitud del modelo de arboles, mientras el modelo de bosque mantiene su pequeña ventaja.

Debería haber un método que permitiera disminuir estas perdidas de exactitud. Por ahora, aunque se intentó alterando algunos otros elementos, no se logra obtener un modelo con una mayor exactitud que estas.

<a id="conclusion"></a>
## Conclusion

Luego de calibrar y comparar nuestros 3 modelos, nos quedamos con 2 posibles opciones:

Arbol de 7 ramas y 13 hojas con 77.3% de exactitud en testeo. Un gran aspecto a su favor es su alta velocidad, como también su exactitud. Aunque la opción 2 tiene una mayor exactitud, esta puede llegar a tomar mucho tiempo de procesamiento. Codigo:

- final_tree = DecisionTreeClassifier(max_depth=7, min_samples_leaf=13, random_state=54321)

Bosque de 3 arboles, 11 ramas y 3 hojas con una exactitud de 78.1% de exactitud en testeo. Este modelo puede tomarse un poco más de tiempo, pero si consideramos grandes escalas de datos, la diferencia en su exactitud con la opción 1 comienza a tomar importancia. Código:

- final_forest = RandomForestClassifier(max_depth=11, min_samples_leaf=3, n_estimators=3, random_state=54321)